In [12]:
import ee

In [14]:
# Initialize the Earth Engine
ee.Initialize()



In [3]:


# Define the area of interest (AOI)
aoi = ee.Geometry.Polygon([
    [39.25879949255623, 21.88984377820288],
    [39.26532262488045, 21.88984377820288],
    [39.26532262488045, 21.894861167298092],
    [39.25879949255623, 21.894861167298092],
    [39.25879949255623, 21.88984377820288]
])#uj

In [6]:
aoi=ee.Geometry.Polygon( [
[46.41388398001249,24.474133938125608],
[46.99272614309843,24.474133938125608],
[46.99272614309843,25.04372308127158],
[46.41388398001249,25.04372308127158],
[46.41388398001249,24.474133938125608]
])  #new RUH-for ndvi values

In [16]:
aoi=ee.Geometry.Polygon( [
[39.230642953025026,21.578023959235942],
[39.237681069480104,21.578023959235942],
[39.237681069480104,21.583052246512302],
[39.230642953025026,21.583052246512302],
[39.230642953025026,21.578023959235942]
]) #alsamer

In [27]:


# Function to mask clouds using the quality band of Landsat 8
def maskL8clouds(image):
    qa = image.select('BQA')
    # Cloud confidence (0-1) and cloud shadow (0-1) bit positions
    cloudBitMask = 1 << 4
    cloudShadowBitMask = 1 << 3
    # Get the pixel QA band
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cloudShadowBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Filter the Landsat 8 ImageCollection for the AOI and date range
startDate = '2021-02-01'
endDate = '2024-3-26'
landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(aoi) \
    .filterDate(startDate, endDate) \
    .map(maskL8clouds)  # Apply the cloud mask to each image

# Print the number of images in the collection
count = landsat8.size().getInfo()
print('Number of images in the collection after filtering: ', count)

# Note: Due to the limitations of GEE and the local environment, the following lines
# demonstrate how you would prepare images for download, but the actual download
# would need to be initiated from the GEE Code Editor Tasks panel.

# Export the image, specifying scale and region.
export = ee.batch.Export.image.toDrive(**{
    'image': landsat8.median(),  # Use median to combine images
    'description': 'Landsat8_alsamer_export',
    'scale': 30,
    'region': aoi
})

# Start the export task
export.start()

# Monitor the task in the Google Earth Engine Code Editor Tasks panel
print('Export task started, check the Tasks tab in GEE Code Editor.')


Number of images in the collection after filtering:  21
Export task started, check the Tasks tab in GEE Code Editor.
